In [2]:
import time
import pandas as pd

from bs4 import BeautifulSoup
import urllib.request as req

import requests
import os
import datetime
import re

In [3]:
import sys
sys.path.append("../")

In [4]:
from index.season.season_index import get_seasons
from product_csv.create_df import game_df

In [5]:
# ディビジョンを選択してください:{v1_m, v2_m, v3_m, v1_w, v2_w}
division = 'v1_m'
# '年度を選択してください:{2019-20, 2018-19, 2017-18...}'
season = '2020-21'
# シーズンを選択してください:{regular, 2019-20(game1, game2, semi_final, final), 2014-15~2018-19(final6, final3, final),
# ~2013-14(semi_final, final)}
s_round = 'regular'
season += '_' + s_round
season

'2020-21_regular'

In [6]:
seasons = get_seasons(division)
print(seasons)

{'301': '2020-21_regular', '283': '2019-20_regular', '296': '2019-20_game1', '297': '2019-20_game2', '298': '2019-20_semi_final', '275': '2019-20_final', '258': '2018-19_regular', '261': '2018-19_final6', '260': '2018-19_final3', '259': '2018-19_final', '241': '2017-18_regular', '239': '2017-18_final6', '240': '2017-18_final3', '242': '2017-18_final', '224': '2016-17_regular', '223': '2016-17_final6', '226': '2016-17_final3', '225': '2016-17_final', '206': '2015-16_regular', '209': '2015-16_final6', '208': '2015-16_final3', '207': '2015-16_final', '181': '2014-15_regular', '184': '2014-15_final6', '183': '2014-15_final3', '182': '2014-15_final', '169': '2013-14_regular', '171': '2013-14_semi_final', '170': '2013-14_final', '156': '2012-13_regular', '158': '2012-13_semi_final', '157': '2012-13_final', '131': '2011-12_regular', '130': '2011-12_semi_final', '129': '2011-12_final', '71': '2010-11_regular', '72': '2010-11_semi_final', '73': '2010-11_final', '28': '2009-10_regular', '29': '2

In [6]:
os.chdir('/Users/sasno/Desktop/MyPandas/vleague')

headers = {"User-Agent": "Mozilla/5.0"}

seasons = get_seasons(division)

if not os.path.isdir(division):
            os.makedirs(division)

os.chdir(division)
print(os.getcwd())

for s_id, s_round in seasons.items():
    print(s_id, s_round)

    sets = ['1','2','3','4','5']
    game_all = None

    # ページ数を取得
    url = 'https://www.vleague.jp/round/list/{}'.format(s_id)
    print(url)
    request = req.Request(url, headers=headers)
    response = req.urlopen(request)
    parse_html = BeautifulSoup(response, 'html.parser')
    hrefs = parse_html.find_all('a', href=re.compile("pg"))

    # for i in hrefs:
    #     print(i.text)
    if len(hrefs) > 0:
        pages = int(hrefs[-2].text)
        print(pages)

        all_list = []
        for pg in range(1,pages+1):
            url = 'https://www.vleague.jp/round/list/{0}?pg={1}'.format(s_id,pg)
            print(url)
            game_df(url,headers,s_round,all_list)
    else:
        all_list = []
        url = 'https://www.vleague.jp/round/list/{}'.format(s_id)
        print(url)
        game_df(url,headers,s_round,all_list)
    try:
        game_all = pd.concat(all_list, ignore_index=True)
        print(game_all)
    except:
        pass
    all_dir = 'all/{}'.format(s_round)
    if not os.path.isdir(all_dir):
        os.makedirs(all_dir)
    
    print(os.getcwd())
    try:
        daily_all = game_all[game_all['名前'] != 'チーム合計']
        print(daily_all)

        data = daily_all
        data.insert(1, '月', data['試合日'].str[5:7])
        data.set_index('試合日')
        monthly_all = data.groupby(['名前','月','チーム','背番号']).sum().reset_index()
        monthly_all['アタック決定率'] = ((monthly_all['アタック得点'] / monthly_all['アタック打数']) * 100).round(1)
        monthly_all['アタック効果率'] = (((monthly_all['アタック得点']-monthly_all['アタック失点']) / monthly_all['アタック打数']) * 100).round(1)
        monthly_all['バックアタック決定率'] = ((monthly_all['バックアタック得点'] / monthly_all['バックアタック打数']) * 100).round(1)
        monthly_all['アタックセット平均'] = (monthly_all['アタック得点'] / monthly_all['出場数']).round(1)
        monthly_all['ブロックセット平均'] = (monthly_all['ブロック得点'] / monthly_all['出場数']).round(1)
        monthly_all['サーブ効果率'] = ((monthly_all['サーブ得点'] * 100 + monthly_all['サーブ効果'] * 25 - monthly_all['サーブ失点'] * 25) / monthly_all['サーブ打数']).round(1)
        monthly_all['サーブレシーブ成功率'] = ((monthly_all['サーブレシーブ成功・優'] * 100 + monthly_all['サーブレシーブ成功・良'] * 50) / monthly_all['受数']).round(1)
        print(monthly_all)

        yearly_all = data.groupby(['名前','チーム','背番号']).sum().reset_index()
        yearly_all['アタック決定率'] = ((yearly_all['アタック得点'] / yearly_all['アタック打数']) * 100).round(1)
        yearly_all['アタック効果率'] = (((yearly_all['アタック得点']-yearly_all['アタック失点']) / yearly_all['アタック打数']) * 100).round(1)
        yearly_all['バックアタック決定率'] = ((yearly_all['バックアタック得点'] / yearly_all['バックアタック打数']) * 100).round(1)
        yearly_all['アタックセット平均'] = (yearly_all['アタック得点'] / yearly_all['出場数']).round(1)
        yearly_all['ブロックセット平均'] = (yearly_all['ブロック得点'] / yearly_all['出場数']).round(1)
        yearly_all['サーブ効果率'] = ((yearly_all['サーブ得点'] * 100 + yearly_all['サーブ効果'] * 25 - yearly_all['サーブ失点'] * 25) / yearly_all['サーブ打数']).round(1)
        yearly_all['サーブレシーブ成功率'] = ((yearly_all['サーブレシーブ成功・優'] * 100 + yearly_all['サーブレシーブ成功・良'] * 50) / yearly_all['受数']).round(1)
        print(yearly_all)

        game_all.to_csv('all/{}/game_all.csv'.format(s_round), index=False, encoding='cp932')
        daily_all.to_csv('all/{}/daily_all.csv'.format(s_round), index=False, encoding='cp932')
        monthly_all.to_csv('all/{}/monthly_all.csv'.format(s_round), index=False, encoding='cp932')
        yearly_all.to_csv('all/{}/yearly_all.csv'.format(s_round), index=False, encoding='cp932')
        print('csv作成成功')
    except:
        pass

  13     119      59      19     49.6   
2           ホフトーマス  03      東レアローズ  19   11      42      23       4     54.8   
3      ルイスアルベルダシルバ  03     堺ブレイザーズ  13   13      74      33      11     44.6   
4            中垣内祐一  03     堺ブレイザーズ   4   13     103      63       8     61.2   
5             中野順平  03      東レアローズ   1    4       0       0       0      NaN   
6             丸茂智彦  03      東レアローズ  12    0       0       0       0      NaN   
7             伊藤信博  03     堺ブレイザーズ  15    4       0       0       0      NaN   
8            佐々木太一  03  サントリーサンバーズ   2   11      45      30       0     66.7   
9             加藤陽一  03      東レアローズ   3   11      74      28       8     37.8   
10            大森陽祐  03  NECブルーロケッツ  15    5       0       0       0      NaN   
11            大竹秀之  03  NECブルーロケッツ   7   13      28      20       1     71.4   
12            大野英洋  03     堺ブレイザーズ  10    4       0       0       0      NaN   
13            大関元樹  03  NECブルーロケッツ   5    0       0       0       0      NaN   

In [11]:
# 2020-21の更新
os.chdir('/Users/sasno/Desktop/MyPandas/vleague')

headers = {"User-Agent": "Mozilla/5.0"}

# seasons = get_seasons(division)


division = 'v1_m'
s_id = "301"
# v1_w
# s_id = "303"
# v2_m
# s_id = "299"
# v2_w
# s_id = "302"
# v3_m
# s_id = "300"
# v3_w
# s_id = "268" 
s_round = "2020-21_regular"

os.chdir(division)
print(os.getcwd())

if not os.path.isdir(division):
            os.makedirs(division)

sets = ['1','2','3','4','5']
game_all = None

# ページ数を取得
url = 'https://www.vleague.jp/round/list/{}'.format(s_id)
print(url)
request = req.Request(url, headers=headers)
response = req.urlopen(request)
parse_html = BeautifulSoup(response, 'html.parser')
hrefs = parse_html.find_all('a', href=re.compile("pg"))

# for i in hrefs:
#     print(i.text)
if len(hrefs) > 0:
    pages = int(hrefs[-2].text)
    print(pages)

    all_list = []
    for pg in range(1,pages+1):
        url = 'https://www.vleague.jp/round/list/{0}?pg={1}'.format(s_id,pg)
        print(url)
        request = req.Request(url, headers=headers)
        response = req.urlopen(request)
        parse_html = BeautifulSoup(response, 'html.parser')
        tables = parse_html.find_all('table')

        url_list = []
        trs = tables[0].find_all('a', href=re.compile("/form/b"))
        for tr in trs:
            game_url = 'https://www.vleague.jp/'+ tr.attrs['href']
            url_list.append(game_url)
        print(url_list)

        for url in url_list:
            # print(url)
            html = requests.get(url, headers=headers)
            check = html.text
            if '試合データがありません' not in check:
                try:
                    data = pd.read_html(check, header=[2])

                    for j in range(2):
                        stats = pd.DataFrame(data[j+4])
                        stats = stats.rename(columns=str)
                        col = stats.columns.values
                        data_item = []
                        for i in col:
                            l = i
                            data_item.append(l)
                        for i in range(1,len(data_item)-2):
                            data_item[-i] = data_item[-i-3]
                            # print(data_item)
                        for i in range(3):
                        #     print(data_item[i])
                            if data_item[i] == '出場数':
                                data_item[i] = '背番号'
                            elif data_item[i] == '1':
                                data_item[i] = 'リベロ'
                            else:
                                data_item[i] = '名前'
                        #     print(i, data_item[i])

                        new_stats = stats.rename(columns={col[i]:data_item[i] for i in range(len(data_item))})
                        new_stats = new_stats.rename(columns={
                            '打数':'アタック打数', '得点':'アタック得点', '失点':'アタック失点', '決定率':'アタック決定率',
                            'セ平ット均':'アタックセット平均','打数.1':'バックアタック打数', '得点.1':'バックアタック得点',
                            '失点.1':'バックアタック失点', '決定率.1':'バックアタック決定率', '得点.2':'ブロック得点',
                            'セ平ット均.1':'ブロックセット平均', '打数.2':'サーブ打数', '得点.3':'サーブ得点', '失点.2':'サーブ失点',
                            '効果':'サーブ効果', '効果率':'サーブ効果率', '成功・優':'サーブレシーブ成功・優', 
                            '成功・良':'サーブレシーブ成功・良', '成功率':'サーブレシーブ成功率'
                            })
                        
                        request = req.Request(url, headers=headers)
                        response = req.urlopen(request)
                        parse_html = BeautifulSoup(response, 'html.parser')
                        table_ha = parse_html.find_all('table')[0]
                        td_ha = table_ha.find_all('td', class_='team')
                        # print(td_ha)
                        team = td_ha[j].text
                        # print('team={}'.format(team))
                        new_stats['アタック決定率'] = new_stats['アタック得点'] / new_stats['アタック打数']
                        new_stats.insert(13, 'アタック効果率',
                            (new_stats['アタック得点'] - new_stats['アタック失点']) / new_stats['アタック打数'])
                        span = parse_html.find_all('span')
                        date = span[1].text.replace('/','-')
                        new_stats.insert(0, '試合日', date)
                        new_stats.insert(1, 'チーム', team)
                        new_stats['名前'] = new_stats['名前'].str.replace('\u3000','')
                        for by_set in sets:
                            new_stats[by_set] = new_stats[by_set].astype(str)
                        # 必要に応じてteamディレクトリを作成
                        team_dir = '{0}/{1}'.format(team,s_round)
                        if not os.path.isdir(team_dir):
                            os.makedirs(team_dir)
                        # new_stats.to_csv('{0}/{1}/{2}.csv'.format(team,s_round,date), index=False, encoding='cp932')
                        # print(new_stats)
                        all_list.append(new_stats)
                except:
                    pass
    game_all = pd.concat(all_list, ignore_index=True)
else:
    all_list = []
    url = 'https://www.vleague.jp/round/list/{}'.format(s_id)
    print(url)
    request = req.Request(url, headers=headers)
    response = req.urlopen(request)
    parse_html = BeautifulSoup(response, 'html.parser')
    tables = parse_html.find_all('table')

    url_list = []
    trs = tables[0].find_all('a', href=re.compile("/form/b"))
    for tr in trs:
        game_url = 'https://www.vleague.jp/'+ tr.attrs['href']
        url_list.append(game_url)
    print(url_list)

    for url in url_list:
        # print(url)
        html = requests.get(url, headers=headers)
        check = html.text
        if '試合データがありません' not in check:
            # try:
            data = pd.read_html(check, header=[2])

            for j in range(2):
                stats = pd.DataFrame(data[j+4])
                stats = stats.rename(columns=str)
                col = stats.columns.values
                data_item = []
                for i in col:
                    l = i
                    data_item.append(l)
                for i in range(1,len(data_item)-2):
                    data_item[-i] = data_item[-i-3]
                    # print(data_item)
                for i in range(3):
                #     print(data_item[i])
                    if data_item[i] == '出場数':
                        data_item[i] = '背番号'
                    elif data_item[i] == '1':
                        data_item[i] = 'リベロ'
                    else:
                        data_item[i] = '名前'
                #     print(i, data_item[i])

                new_stats = stats.rename(columns={col[i]:data_item[i] for i in range(len(data_item))})
                new_stats = new_stats.rename(columns={
                    '打数':'アタック打数', '得点':'アタック得点', '失点':'アタック失点', '決定率':'アタック決定率',
                    'セ平ット均':'アタックセット平均','打数.1':'バックアタック打数', '得点.1':'バックアタック得点',
                    '失点.1':'バックアタック失点', '決定率.1':'バックアタック決定率', '得点.2':'ブロック得点',
                    'セ平ット均.1':'ブロックセット平均', '打数.2':'サーブ打数', '得点.3':'サーブ得点', '失点.2':'サーブ失点',
                    '効果':'サーブ効果', '効果率':'サーブ効果率', '成功・優':'サーブレシーブ成功・優', 
                    '成功・良':'サーブレシーブ成功・良', '成功率':'サーブレシーブ成功率'
                    })
                
                request = req.Request(url, headers=headers)
                response = req.urlopen(request)
                parse_html = BeautifulSoup(response, 'html.parser')
                table_ha = parse_html.find_all('table')[0]
                td_ha = table_ha.find_all('td', class_='team')
                # print(td_ha)
                team = td_ha[j].text
                # print('team={}'.format(team))
                new_stats['アタック決定率'] = (new_stats['アタック得点'] / new_stats['アタック打数']).round(1)
                new_stats.insert(13, 'アタック効果率', (new_stats['アタック得点'] - new_stats['アタック失点']) / new_stats['アタック打数'].round(1))
                span = parse_html.find_all('span')
                date = span[1].text.replace('/','-')
                new_stats.insert(0, '試合日', date)
                new_stats.insert(1, 'チーム', team)
                new_stats['名前'] = new_stats['名前'].str.replace('\u3000','')
                for by_set in sets:
                    new_stats[by_set] = new_stats[by_set].astype(str)
                必要に応じてteamディレクトリを作成
                team_dir = '{0}/{1}'.format(team,s_round)
                if not os.path.isdir(team_dir):
                    os.makedirs(team_dir)
                new_stats.to_csv('{0}/{1}/{2}.csv'.format(team,s_round,date), index=False, encoding='cp932')
                all_list.append(new_stats)
            # except:
            #     pass
    try:
        game_all = pd.concat(all_list, ignore_index=True)
        print(game_all)

    except:
        pass
all_dir = 'all/{}'.format(s_round)
if not os.path.isdir(all_dir):
    os.makedirs(all_dir)

print(os.getcwd())
try:
    daily_all = game_all[game_all['名前'] != 'チーム合計']
    print(daily_all)

    data = daily_all
    data.insert(1, '月', data['試合日'].str[5:7])
    data.set_index('試合日')
    monthly_all = data.groupby(['名前','月','チーム','背番号']).sum().reset_index()
    monthly_all['アタック決定率'] = ((monthly_all['アタック得点'] / monthly_all['アタック打数']) * 100).round(1)
    monthly_all['アタック効果率'] = (((monthly_all['アタック得点']-monthly_all['アタック失点']) / monthly_all['アタック打数']) * 100).round(1)
    monthly_all['バックアタック決定率'] = ((monthly_all['バックアタック得点'] / monthly_all['バックアタック打数']) * 100).round(1)
    monthly_all['アタックセット平均'] = (monthly_all['アタック得点'] / monthly_all['出場数']).round(2)
    monthly_all['ブロックセット平均'] = (monthly_all['ブロック得点'] / monthly_all['出場数']).round(2)
    monthly_all['サーブ効果率'] = ((monthly_all['サーブ得点'] * 100 + monthly_all['サーブ効果'] * 25 - monthly_all['サーブ失点'] * 25) / monthly_all['サーブ打数']).round(1)
    monthly_all['サーブレシーブ成功率'] = ((monthly_all['サーブレシーブ成功・優'] * 100 + monthly_all['サーブレシーブ成功・良'] * 50) / monthly_all['受数']).round(1)
    monthly_all['フロントアタック打数'] = monthly_all['アタック打数'] - monthly_all['バックアタック打数']
    monthly_all['総得点'] = monthly_all['アタック得点'] + monthly_all['ブロック得点'] + monthly_all['サーブ得点']
    print(monthly_all)

    yearly_all = data.groupby(['名前','チーム','背番号']).sum().reset_index()
    yearly_all['アタック決定率'] = ((yearly_all['アタック得点'] / yearly_all['アタック打数']) * 100).round(1)
    yearly_all['アタック効果率'] = (((yearly_all['アタック得点']-yearly_all['アタック失点']) / yearly_all['アタック打数']) * 100).round(1)
    yearly_all['バックアタック決定率'] = ((yearly_all['バックアタック得点'] / yearly_all['バックアタック打数']) * 100).round(1)
    yearly_all['アタックセット平均'] = (yearly_all['アタック得点'] / yearly_all['出場数']).round(2)
    yearly_all['ブロックセット平均'] = (yearly_all['ブロック得点'] / yearly_all['出場数']).round(2)
    yearly_all['サーブ効果率'] = ((yearly_all['サーブ得点'] * 100 + yearly_all['サーブ効果'] * 25 - yearly_all['サーブ失点'] * 25) / yearly_all['サーブ打数']).round(1)
    yearly_all['サーブレシーブ成功率'] = ((yearly_all['サーブレシーブ成功・優'] * 100 + yearly_all['サーブレシーブ成功・良'] * 50) / yearly_all['受数']).round(1)
    yearly_all['総得点'] = yearly_all['アタック得点'] + yearly_all['ブロック得点'] + yearly_all['サーブ得点']
    print(yearly_all)

    game_all.to_csv('all/{}/game_all.csv'.format(s_round), index=False)
    daily_all.to_csv('all/{}/daily_all.csv'.format(s_round), index=False)
    monthly_all.to_csv('all/{}/monthly_all.csv'.format(s_round), index=False)
    yearly_all.to_csv('all/{}/yearly_all.csv'.format(s_round), index=False)
    print('csv作成成功')
except:
    pass

c:\Users\sasno\Desktop\MyPandas\vleague\v1_m
https://www.vleague.jp/round/list/301
9
https://www.vleague.jp/round/list/301?pg=1
['https://www.vleague.jp//form/b/25724', 'https://www.vleague.jp//form/b/25722', 'https://www.vleague.jp//form/b/25716', 'https://www.vleague.jp//form/b/25718', 'https://www.vleague.jp//form/b/25720', 'https://www.vleague.jp//form/b/25717', 'https://www.vleague.jp//form/b/25721', 'https://www.vleague.jp//form/b/25725', 'https://www.vleague.jp//form/b/25723', 'https://www.vleague.jp//form/b/25719', 'https://www.vleague.jp//form/b/25726', 'https://www.vleague.jp//form/b/25732', 'https://www.vleague.jp//form/b/25728', 'https://www.vleague.jp//form/b/25734', 'https://www.vleague.jp//form/b/25727', 'https://www.vleague.jp//form/b/25729', 'https://www.vleague.jp//form/b/25733', 'https://www.vleague.jp//form/b/25735', 'https://www.vleague.jp//form/b/25730', 'https://www.vleague.jp//form/b/25731']
https://www.vleague.jp/round/list/301?pg=2
['https://www.vleague.jp//fo

In [ ]:
# 日別・月別成績の作成
daily_all = game_all[game_all['名前']!='チーム合計']
data = daily_all
data.insert(1, '月', data['試合日'].str[5:7])
data.set_index('試合日')
monthly_all = data.groupby(['名前','月','チーム','背番号']).sum().reset_index()
monthly_all['アタック決定率'] = ((monthly_all['アタック得点'] / monthly_all['アタック打数']) * 100).round(1)
monthly_all['アタック効果率'] = (((monthly_all['アタック得点']-monthly_all['アタック失点']) / monthly_all['アタック打数']) * 100).round(1)
monthly_all['バックアタック決定率'] = ((monthly_all['バックアタック得点'] / monthly_all['バックアタック打数']) * 100).round(1)
monthly_all['アタックセット平均'] = (monthly_all['アタック得点'] / monthly_all['出場数']).round(2)
monthly_all['ブロックセット平均'] = (monthly_all['ブロック得点'] / monthly_all['出場数']).round(2)
monthly_all['サーブ効果率'] = ((monthly_all['サーブ得点'] * 100 + monthly_all['サーブ効果'] * 25 - monthly_all['サーブ失点'] * 25) / monthly_all['サーブ打数']).round(1)
monthly_all['サーブレシーブ成功率'] = ((monthly_all['サーブレシーブ成功・優'] * 100 + monthly_all['サーブレシーブ成功・良'] * 50) / monthly_all['受数']).round(1)
monthly_all

In [7]:
# 年別成績の作成
daily_all = game_all[game_all['名前']!='チーム合計']
data = daily_all
data.insert(1, '月', data['試合日'].str[5:7])
data.set_index('試合日')
yearly_all = data.groupby(['名前','チーム','背番号']).sum().reset_index()
yearly_all['アタック決定率'] = ((yearly_all['アタック得点'] / yearly_all['アタック打数']) * 100).round(1)
yearly_all['アタック効果率'] = (((yearly_all['アタック得点']-yearly_all['アタック失点']) / yearly_all['アタック打数']) * 100).round(1)
yearly_all['バックアタック決定率'] = ((yearly_all['バックアタック得点'] / yearly_all['バックアタック打数']) * 100).round(1)
yearly_all['アタックセット平均'] = (yearly_all['アタック得点'] / yearly_all['出場数']).round(2)
yearly_all['ブロックセット平均'] = (yearly_all['ブロック得点'] / yearly_all['出場数']).round(2)
yearly_all['サーブ効果率'] = ((yearly_all['サーブ得点'] * 100 + yearly_all['サーブ効果'] * 25 - yearly_all['サーブ失点'] * 25) / yearly_all['サーブ打数']).round(1)
yearly_all['サーブレシーブ成功率'] = ((yearly_all['サーブレシーブ成功・優'] * 100 + yearly_all['サーブレシーブ成功・良'] * 50) / yearly_all['受数']).round(1)
yearly_all.dtypes

名前              object
チーム             object
背番号             object
出場数              int64
アタック打数           int64
アタック得点           int64
アタック失点           int64
アタック決定率        float64
アタック効果率        float64
バックアタック打数        int64
バックアタック得点        int64
バックアタック失点        int64
ブロック得点           int64
サーブ打数            int64
サーブ得点            int64
サーブ失点            int64
サーブ効果            int64
受数               int64
サーブレシーブ成功・優      int64
サーブレシーブ成功・良      int64
バックアタック決定率     float64
アタックセット平均      float64
ブロックセット平均      float64
サーブ効果率         float64
サーブレシーブ成功率     float64
dtype: object

In [ ]:
all_dir = 'all/{}'.format(season)
if not os.path.isdir(all_dir):
        os.makedirs(all_dir)

In [ ]:
game_all = pd.concat(all_list, ignore_index=True)
# game_all
game_all.to_csv('all/{}/game_all.csv'.format(season), index=False, encoding='cp932')

In [ ]:
os.chdir('/Users/sasno/Desktop/MyPandas/vleague')

headers = {"User-Agent": "Mozilla/5.0"}

seasons = get_seasons(season,division)

for s_id, s_round in seasons.items():
    if season == s_round:
        season_id = s_id
print(season_id, season)
if not os.path.isdir(division):
            os.makedirs(division)

os.chdir(division)
print(os.getcwd())

sets = ['1','2','3','4','5']

# ページ数を取得
url = 'https://www.vleague.jp/round/list/{}'.format(season_id)
print(url)
request = req.Request(url, headers=headers)
response = req.urlopen(request)
parse_html = BeautifulSoup(response, 'html.parser')
hrefs = parse_html.find_all('a', href=re.compile("pg"))

# for i in hrefs:
#     print(i.text)
pages = int(hrefs[-2].text)
print(pages)

all_list = []
for pg in range(1,pages+1):
    url = 'https://www.vleague.jp/round/list/{0}?pg={1}'.format(season_id,pg)
    print(url)
    request = req.Request(url, headers=headers)
    response = req.urlopen(request)
    parse_html = BeautifulSoup(response, 'html.parser')
    tables = parse_html.find_all('table')

    url_list = []
    tr = tables[0].find_all('a')
    for i in range(1,len(tr),2):
        game_url = 'https://www.vleague.jp/'+ tr[i].attrs['href']
        url_list.append(game_url)

    for url in url_list:
        # print(url)
        html = requests.get(url, headers=headers)
        data = pd.read_html(html.text, header=[2])

        for j in range(2):
            stats = pd.DataFrame(data[j+4])
            stats = stats.rename(columns=str)
            col = stats.columns.values
            data_item = []
            for i in col:
                l = i
                data_item.append(l)
            for i in range(1,len(data_item)-2):
                data_item[-i] = data_item[-i-3]
                # print(data_item)
            for i in range(3):
            #     print(data_item[i])
                if data_item[i] == '出場数':
                    data_item[i] = '背番号'
                elif data_item[i] == '1':
                    data_item[i] = 'リベロ'
                else:
                    data_item[i] = '名前'
            #     print(i, data_item[i])

            new_stats = stats.rename(columns={col[i]:data_item[i] for i in range(len(data_item))})
            new_stats = new_stats.rename(columns={
                '打数':'アタック打数', '得点':'アタック得点', '失点':'アタック失点', '決定率':'アタック決定率',
                'セ平ット均':'アタックセット平均','打数.1':'バックアタック打数', '得点.1':'バックアタック得点',
                '失点.1':'バックアタック失点', '決定率.1':'バックアタック決定率', '得点.2':'ブロック得点',
                'セ平ット均.1':'ブロックセット平均', '打数.2':'サーブ打数', '得点.3':'サーブ得点', '失点.2':'サーブ失点',
                '効果':'サーブ効果', '効果率':'サーブ効果率', '成功・優':'サーブレシーブ成功・優', 
                '成功・良':'サーブレシーブ成功・良', '成功率':'サーブレシーブ成功率'
                })
            
            request = req.Request(url, headers=headers)
            response = req.urlopen(request)
            parse_html = BeautifulSoup(response, 'html.parser')
            table_ha = parse_html.find_all('table')[0]
            td_ha = table_ha.find_all('td', class_='team')
            # print(td_ha)
            team = td_ha[j].text
            # print('team={}'.format(team))
            new_stats['アタック決定率'] = new_stats['アタック得点'] / new_stats['アタック打数']
            new_stats.insert(13, 'アタック効果率',
                (new_stats['アタック得点'] - new_stats['アタック失点']) / new_stats['アタック打数'])
            span = parse_html.find_all('span')
            date = span[1].text.replace('/','-')
            new_stats.insert(0, '試合日', date)
            new_stats.insert(1, 'チーム', team)
            new_stats['名前'] = new_stats['名前'].str.replace('\u3000','')
            for by_set in sets:
                new_stats[by_set] = new_stats[by_set].astype(str)
            team_dir = '{0}/{1}'.format(team,season)
            if not os.path.isdir(team_dir):
                os.makedirs(team_dir)
            # new_stats.to_csv('{0}/{1}/{2}.csv'.format(team,season,date), index=False, encoding='cp932')
            all_list.append(new_stats)

In [ ]:
# ページ数を取得
url = 'https://www.vleague.jp/round/list/{}'.format(division_id)
print(url)
request = req.Request(url, headers=headers)
response = req.urlopen(request)
parse_html = BeautifulSoup(response, 'html.parser')
href = parse_html.find_all('a', href=re.compile("pg"))

In [5]:
set_l = [1,2,3,4]
j = 1
set_l[-(j+2)]

2